In [56]:
# Para importar la base de datos leucemia
from sklearn.datasets.mldata import fetch_mldata
import tempfile

import numpy as np

# Para Procesamiento de datos
## Regresion logistica
from sklearn.linear_model import LogisticRegression   #Importa las fxn de validacion cruzadas
from sklearn.cross_validation import train_test_split #Importa las funciones de validación cruzada
from sklearn.preprocessing import StandardScaler      #Importar las funciones de preparacion 
## Naive bayes
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB #importar libreria del clasificador 
## Clustering
# K-Means
from sklearn.cluster import KMeans
from sklearn import cluster
## GMM
from matplotlib import patches # es para hacer elipses
from sklearn import datasets
from sklearn.mixture import GMM #mixture contiene los modelos de mezclas
from sklearn.cross_validation import StratifiedKFold #validación por K-folds
## Arboles 
from sklearn.tree import DecisionTreeClassifier
from sklearn import cross_validation
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

In [2]:
test_data_home = tempfile.mkdtemp()
leuk = fetch_mldata('leukemia', transpose_data=True, data_home=test_data_home)

X = leuk['data']

y = leuk['target']


# Analisis Supervisados

In [6]:
#__________________________________ Preprocesamiento de datos ___________________________________#
clasificador = LogisticRegression(C=1000.0,random_state=0) # C es el parametro
clasificador.fit(x_norm_train, y_train) # entrenamiento del clasificador

# para validar el clasificador
y_pred = clasificador.predict(x_norm_test)

print('Las muestras mal clasificadas fueron de %d/%d '%((y_test == y_pred).sum(),len(y_pred)))

validar_clasificador(y_train,X_train,clasificador,y_test,y_pred)

Las muestras mal clasificadas fueron de 21/22 

######################################################################

 Desempeño del clasificador sobre el conjunto de entrenamiento 

               precision    recall  f1-score   support

Primera Clase       1.00      1.00      1.00        16
Segunda Clase       1.00      1.00      1.00        34

  avg / total       1.00      1.00      1.00        50

######################################################################


######################################################################
Desempeño del clasificador sobre el conjunto de la validación 

               precision    recall  f1-score   support

Primera Clase       0.90      1.00      0.95         9
Segunda Clase       1.00      0.92      0.96        13

  avg / total       0.96      0.95      0.95        22


######################################################################


In [7]:
#____________________________ Naive Bayes ____________________________________#

#generar el clasificador
clasificador = GaussianNB() #instanciamos y luego entrenamos
clasificador.fit(x_norm_train, y_train) #entrenamos el clasificador

#hacer la predicion
y_predict = clasificador.predict(x_norm_test)

print('Las muestras mal clasificadas fueron de %d/%d '%((y_test == y_predict).sum(),len(y_predict)))


validar_clasificador(y_train,X_train,clasificador,y_test,y_predict)

Las muestras mal clasificadas fueron de 22/22 

######################################################################

 Desempeño del clasificador sobre el conjunto de entrenamiento 

               precision    recall  f1-score   support

Primera Clase       1.00      1.00      1.00        16
Segunda Clase       1.00      1.00      1.00        34

  avg / total       1.00      1.00      1.00        50

######################################################################


######################################################################
Desempeño del clasificador sobre el conjunto de la validación 

               precision    recall  f1-score   support

Primera Clase       0.90      1.00      0.95         9
Segunda Clase       1.00      0.92      0.96        13

  avg / total       0.96      0.95      0.95        22


######################################################################


# Aprendizaje No Supervisado

In [36]:
#__________________________ K-Means ______________________________________#

# definimos el número de clusters o grupos
num_cluster = 2
#generamos el modelo 

#init establece el método de localizacion inicial de los centroides
modelo_kmeans= KMeans(init = 'k-means++',n_clusters = num_cluster, n_init=10)

#entrenar el modelo 
modelo_kmeans.fit(X)

#centros
centroides = modelo_kmeans.cluster_centers_

labels = modelo_kmeans.labels_ 
print('Etiquetas con Clustering : ')
print('______________________________________________________________________________')
print(labels)
print('Etiquetas originales: ')
print('______________________________________________________________________________')
print(y)
clase_0 = 0
clase_1 = 0

labels_t = np.where(labels==0,-1,labels)
print('______________________________________________________________________________')
print(labels_t)
cont = 0
for i in range(len(labels)):
    if labels_t[i] == labels[i]:
        cont = cont + 1
val = (cont/len(labels))*100
print('El porcentaje de muestras bien clasificadas es de : ',val,'%')

Etiquetas con Clustering : 
______________________________________________________________________________
[1 1 1 1 1 0 0 0 1 0 1 0 1 1 1 1 1 0 0 1 0 0 0 1 0 1 0 1 0 1 0 1 1 1 0 1 1
 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 1]
Etiquetas originales: 
______________________________________________________________________________
[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1  1  1  1  1  1  1  1  1  1  1 -1
 -1 -1 -1 -1  1  1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1  1  1  1  1  1  1]
______________________________________________________________________________
[ 1  1  1  1  1 -1 -1 -1  1 -1  1 -1  1  1  1  1  1 -1 -1  1 -1 -1 -1  1 -1
  1 -1  1 -1  1 -1  1  1  1 -1  1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1  1
 -1 -1  1  1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1  1  1 -1 -1  1]
El porcentaje de muestras bien clasificadas es de :  41.66666666666667 %


# Modelos de mezclas Gaussianas GMM

In [47]:
y_respaldo = y
y_respaldo = np.where(y_respaldo==-1,0,y_respaldo)

#print(y_respaldo)

#realizamos las particiones de los datos para entrenamiento y validacion 80/20
indices = StratifiedKFold(y_respaldo, n_folds = 5)

#objetos iterables para sacar los indices
train_index,test_index = next(iter(indices))

#extraer los datos del contenido de iris
X_train = X[train_index]
y_train = y_respaldo[train_index]
X_test = X[test_index]
y_test = y_respaldo[test_index]


#calculamos el número de clases
numero_clases = len(np.unique(y_train))

# Construir el clasificador GMM
clasificador = GMM(n_components = numero_clases, covariance_type = 'full', init_params = 'w', n_iter = 20)

clasificador.means_ = np.array([X_train[y_train==i].mean(axis=0)for i in range(numero_clases)])


#Entrenamos con fit
clasificador.fit(X_train)
y_train_predict = clasificador.predict(X_train)
acc_train = np.mean(y_train_predict.ravel() == y_train.ravel())*100
print('El acierto de entrenamiento es del ',acc_train,'%')

y_test_predict = clasificador.predict(X_test)
acc_test = np.mean(y_test_predict.ravel() == y_test.ravel())*100
print('El acierto de entrenamiento es del ',acc_test,'%')

# Reporte de validación
validar_clasificador(y_train,X_train,clasificador,y_test,y_test_predict)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:52: DeprecationWarning: Class GMM is deprecated; The class GMM is deprecated in 0.18 and will be  removed in 0.20. Use class GaussianMixture instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function distribute_covar_matrix_to_match_covariance_type is deprecated; The functon distribute_covar_matrix_to_match_covariance_typeis deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Func

El acierto de entrenamiento es del  100.0 %


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)


El acierto de entrenamiento es del  100.0 %

######################################################################

 Desempeño del clasificador sobre el conjunto de entrenamiento 



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)


               precision    recall  f1-score   support

Primera Clase       1.00      1.00      1.00        20
Segunda Clase       1.00      1.00      1.00        37

  avg / total       1.00      1.00      1.00        57

######################################################################


######################################################################
Desempeño del clasificador sobre el conjunto de la validación 

               precision    recall  f1-score   support

Primera Clase       1.00      1.00      1.00         5
Segunda Clase       1.00      1.00      1.00        10

  avg / total       1.00      1.00      1.00        15


######################################################################


# Arboles de decisiones

In [52]:
# Separamos los datos de acuerdo a las etiquetas(2 clases
clase1 = np.array(X[y==1])
clase_1 = np.array(X[y==-1])

# Dividimos los datos en entrenamiento y validación
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X,y, test_size = 0.25, random_state = 5)

# construimos el clasificador
# variable tipo diccionario con los parametros del arbol
# random state: semilla con la que arranca las cosas aleatorias
# max_depth: Numero de niveles del arbolito.
parametros = {'random_state' : 0, 'max_depth':4}
#construir arbol de clasificación
clasificador = DecisionTreeClassifier(**parametros)

# lo entrenamos para que con el conjunto de entrenamiento se arme el arbolito
clasificador.fit(X_train, y_train)

y_test_pred = clasificador.predict(X_test)

validar_clasificador(y_train,X_train,clasificador,y_test,y_test_pred)


######################################################################

 Desempeño del clasificador sobre el conjunto de entrenamiento 

               precision    recall  f1-score   support

Primera Clase       1.00      1.00      1.00        19
Segunda Clase       1.00      1.00      1.00        35

  avg / total       1.00      1.00      1.00        54

######################################################################


######################################################################
Desempeño del clasificador sobre el conjunto de la validación 

               precision    recall  f1-score   support

Primera Clase       1.00      0.83      0.91         6
Segunda Clase       0.92      1.00      0.96        12

  avg / total       0.95      0.94      0.94        18


######################################################################


# Bosques aleatorios

In [58]:

tipo_clasificador = 'ba'

parametros = { 'n_estimators':100, 'max_depth': 4, 'random_state':0}

if tipo_clasificador == 'ba':
    # Bosque aleatorio
    clasificador = RandomForestClassifier(**parametros)
else:
    # Bosque extra aleatorio
    clasificador = ExtraTreesClassifier(**parametros)

# Entrenamos el clasificador
clasificador.fit(X_train, y_train)

#validamos el clasificador
y_test_pred = clasificador.predict(X_test)


## REPORTE DE CLASIFICACIÓN
validar_clasificador(y_train,X_train,clasificador,y_test,y_test_pred)


######################################################################

 Desempeño del clasificador sobre el conjunto de entrenamiento 

               precision    recall  f1-score   support

Primera Clase       1.00      1.00      1.00        19
Segunda Clase       1.00      1.00      1.00        35

  avg / total       1.00      1.00      1.00        54

######################################################################


######################################################################
Desempeño del clasificador sobre el conjunto de la validación 

               precision    recall  f1-score   support

Primera Clase       1.00      1.00      1.00         6
Segunda Clase       1.00      1.00      1.00        12

  avg / total       1.00      1.00      1.00        18


######################################################################


In [46]:
def validar_clasificador(y_train,X_train,clasificador,y_test,y_predict):
    # Evaluamos el desempeño del clasificador imprimiendo el reporte de clasificación.
    nombres_clases = ['Primera Clase','Segunda Clase']
    print('\n' + '#'*70)
    print('\n Desempeño del clasificador sobre el conjunto de entrenamiento \n')
    print(classification_report(y_train, clasificador.predict(X_train), target_names = nombres_clases))
    print('#'*70 + '\n')
    print('\n'+ '#'*70 )
    print('Desempeño del clasificador sobre el conjunto de la validación \n')
    print(classification_report(y_test,y_predict, target_names = nombres_clases))
    print('\n'+ '#'*70 )

In [4]:
# Remocion de la media
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.3, random_state = 3)

#Remoción para x_train
def Remocion(X):
    med = X.mean(axis=0)
    des = X.std(axis=0)
    X = X - med
    X = X/des
    return X,med, des

#Remocion para x_test
def Remocion_test(X,med,des):
    X = X - med
    X = X/des
    return X

#Remocion para x_train
x_norm_train,t_train_mean,t_train_desv = Remocion(X_train)
#Remocion para x_test
x_norm_test = Remocion_test(X_test,t_train_mean,t_train_desv)